Para el entrenamiento NER necesito el dataset con un formato diferente. Usaremos los mismos datos, y las preguntas nos servirán para identificar cada tipo de dato. Protocolos, Fechas, Nombres de notario y Tipos de documento.

In [1]:
import json
import os
import pandas as pd
from globals import DATA_DIR, TIPO_ETIQUETA, Tipo_Contenido, label2id, id2label
from sklearn.model_selection import train_test_split

In [2]:
def cargar_datos(archivo_json):
    with open(archivo_json, 'r', encoding='utf-8') as f:
        return json.load(f)['data']
    
def etiqueta_tipo(tipo:Tipo_Contenido):
    """Indica qué tipo de etiqueta es este tipo. Sólo sufijo de la etiqueta, pues no se conoce la posición."""
    try:
        return TIPO_ETIQUETA[tipo]    
    except:
        return None

def generar_ner_dataset(data):
    dataset_ner = []
    for item in data:
        for paragraph in item['paragraphs']:
            id = paragraph['id']
            context = paragraph['context']
            tokens = context.split()
            labels = ['O'] * len(tokens) 
            for qa in paragraph['qas']:
                question = qa['question']
                tipo = etiqueta_tipo(Tipo_Contenido.tipo(question))
                for answer in qa['answers']:                    
                    answer_start = answer['answer_start']
                    answer_tokens = answer['text'].split()                    
                    # Calcular índices de inicio y fin en tokens
                    start_index = len(context[:answer_start].split())
                    # es uno más, para que funcione el range
                    end_index = start_index + len(answer_tokens)

                    # Actualizar labels para esta respuesta
                    for i in range(start_index, end_index):
                        tag = 'B-' + tipo if i == start_index else 'I-' + tipo
                        labels[i] = tag
            ner_tags = [label2id[label] for label in labels]
            dataset_ner.append({'id': id, 'tokens': tokens, 'ner_tags': ner_tags, 'labels': labels})
    return dataset_ner

In [4]:
# Cargar los datos y generar splits
archivo_json = os.path.join(DATA_DIR, 'dataset_QA_ANONIMIZADO.json')
datos_json = cargar_datos(archivo_json)
ner_dataset = generar_ner_dataset(datos_json)
train_val, test_data = train_test_split(ner_dataset, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_val, test_size=0.2, random_state=42)
# Guardar datasets
archivos = [('train', train_data), ('validation', val_data), ('test', test_data)]
for nombre, data in archivos:
    with open(os.path.join(DATA_DIR,'ESCRITURAS', f'NER_{nombre}.json'), 'w', encoding='utf-8') as f:
        json.dump(data,f,ensure_ascii=False)